In [1]:
import json
from utils import get_distance_between_points
import geopandas as gpd
from shapely import Point

# Valencia region
valencia_region_polygon = gpd.read_file("./data/valencia_region.geojson").dissolve()[
    "geometry"
][0]

# Load input data
possible_locations = []
with open("./data/possible_locations.json") as f:
    possible_locations = json.load(f)

# Load oil containers data (to get current locations)
oil_containers = []
with open("./data/contenidors-oli-usat-contenedores-aceite-usado.json") as f:
    oil_containers = json.load(f)

oil_containers_locations = []
# For each container
for container in oil_containers:
    # Get location
    location = [container["geo_point_2d"]["lon"], container["geo_point_2d"]["lat"]]

    # Check if locations is in valencia region
    if valencia_region_polygon.contains(Point(location)):
        # Then, append
        oil_containers_locations.append(location)


def find_closest_unique_pairs(a, b):
    """Find the closest unique pairs of coordinates between two lists."""
    results = {}
    used_b_indices = set()

    for i, coord_a in enumerate(a):
        min_distance = float("inf")
        closest_index = None

        for j, coord_b in enumerate(b):
            if j in used_b_indices:
                continue

            distance = get_distance_between_points(coord_a, coord_b)

            if distance < min_distance:
                min_distance = distance
                closest_index = j

        if closest_index is not None:
            results[i] = {
                "a_coord": coord_a,
                "b_index": closest_index,
                "b_coord": b[closest_index],
                "distance": min_distance,
            }
            used_b_indices.add(closest_index)

    return results


# Encontrar los emparejamientos únicos más cercanos
results = find_closest_unique_pairs(oil_containers_locations, possible_locations)

# Imprimir los resultados
for key, value in results.items():
    print(f"Lista a[{key}] emparejada con Lista b[{value['b_index']}]:")
    print(
        f"  a = {value['a_coord']}, b = {value['b_coord']}, distancia = {value['distance']} m\n"
    )


Lista a[0] emparejada con Lista b[3033]:
  a = [-0.3834991044482499, 39.478164703085184], b = [-0.3834593298982603, 39.47827432112632], distancia = 12.642301276125163 m

Lista a[1] emparejada con Lista b[4135]:
  a = [-0.3520073459198504, 39.46112152046048], b = [-0.3520511636679707, 39.46076896947802], distancia = 39.32295906244833 m

Lista a[2] emparejada con Lista b[4266]:
  a = [-0.3927993965757745, 39.48700952293332], b = [-0.3928852694037598, 39.48736945934605], distancia = 40.638983324089935 m

Lista a[3] emparejada con Lista b[2804]:
  a = [-0.3772980467911945, 39.46808530153522], b = [-0.37715077374866607, 39.46806282595864], distancia = 12.916578844187795 m

Lista a[4] emparejada con Lista b[5130]:
  a = [-0.3883010517909016, 39.49895515142951], b = [-0.3886930171811705, 39.49906395067738], distancia = 35.813377513558265 m

Lista a[5] emparejada con Lista b[977]:
  a = [-0.3611717379539136, 39.47364910567398], b = [-0.36164025976259406, 39.47370060767262], distancia = 40.7178

In [26]:
import pandas as pd

df = pd.DataFrame(results)
df = df.transpose()
df.to_csv("mapeado.csv")
df[df.distance > 100]

,a_coord,b_index,b_coord,distance
110,"[-0.3974687920139127, 39.486880824869445]",4341,"[-0.397790081045588, 39.48592889773259]",109.242152
238,"[-0.3767966713241999, 39.46628925527343]",64,"[-0.37624814659460876, 39.46544724116372]",104.725606
242,"[-0.40841802232151014, 39.47269409722403]",389,"[-0.4098536073985222, 39.472537268500986]",124.749125


In [22]:
df.sort_values(by=['distance'])

,a_coord,b_index,b_coord,distance
78,"[-0.36897772040251403, 39.468969724859335]",3568,"[-0.3689713287367715, 39.468965439713045]",0.727227
285,"[-0.3708722843650569, 39.45044211705346]",2971,"[-0.37086849447726883, 39.45045210497884]",1.155887
329,"[-0.3726949397019562, 39.49443549469304]",1138,"[-0.37270608878734046, 39.49444186644522]",1.19173
139,"[-0.3519017200729492, 39.47472814285904]",2529,"[-0.3519065355548746, 39.47471592861362]",1.41797
300,"[-0.3735831992277223, 39.453284473160494]",3283,"[-0.3735991437277687, 39.45327937683228]",1.484424
...,...,...,...,...
20,"[-0.3426772837324609, 39.45974636517033]",2219,"[-0.343277521299238, 39.45907048707039]",91.101222
187,"[-0.35652775940254716, 39.4710914524283]",2035,"[-0.3576498636669515, 39.47117658315028]",97.017428
238,"[-0.3767966713241999, 39.46628925527343]",64,"[-0.37624814659460876, 39.46544724116372]",104.725606
110,"[-0.3974687920139127, 39.486880824869445]",4341,"[-0.397790081045588, 39.48592889773259]",109.242152


In [25]:
df.loc[:, 'distance'].mean()


30.154486035339463